In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backpor

In [ ]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2021-12-16 03:47:49--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  5.91MB/s    in 0.2s    

2021-12-16 03:47:50 (5.91 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

### Load Amazon Data into Spark DataFrame

In [3]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Digital_Video_Games_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
# Show DataFrame
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+-------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|   product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+-------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   21269168| RSH1OZ87OYK92|B013PURRZW|     603406193|Madden NFL 16 - X...|Digital_Video_Games|          2|            2|          3|   N|                N|A slight improvem...|I keep buying mad...| 2015-08-31|
|         US|     133437|R1WFOQ3N9BO65I|B00F4CEHNK|     341969535| Xbox Live Gift Card|Digital_Video_Games| 

### Create DataFrames to match tables

In [4]:
vine_df = df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])
vine_df.show(20)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RSH1OZ87OYK92|          2|            2|          3|   N|                N|
|R1WFOQ3N9BO65I|          5|            0|          0|   N|                Y|
| R3YOOS71KM5M9|          5|            0|          0|   N|                Y|
|R3R14UATT3OUFU|          5|            0|          0|   N|                Y|
| RV2W9SGDNQA2C|          5|            0|          0|   N|                Y|
|R3CFKLIZ0I2KOB|          5|            0|          0|   N|                Y|
|R1LRYU1V0T3O38|          4|            0|          0|   N|                Y|
| R44QKV6FE5CJ2|          5|            0|          0|   N|                Y|
|R2TX1KLPXXXNYS|          5|            0|          0|   N|                Y|
|R1JEEW4C6R89BA|          5|            0|          0|   N|     

In [5]:
# Review Data Types
from pyspark.sql.types import IntegerType
vine_df.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)



In [6]:
# 1) Create a new DataFrame or table to retrieve all the rows where the number of helpful_votes divided by total_votes
# is equal to or greater than 50%.
vine_df_filtered = vine_df.filter("total_votes>20")
vine_df_filtered.show(20)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RV4UH0OWW97K4|          5|           19|         22|   N|                N|
| RG6GT6G9WNYT1|          5|           22|         22|   N|                N|
| RFBXE3Q3GYQYO|          5|           68|         70|   N|                Y|
|R235HEIYY15NZX|          1|            0|         26|   N|                Y|
|R3BZ4DHX2YHC4V|          1|            1|         32|   N|                Y|
| RZSC8Q4630BV1|          1|            1|         26|   N|                N|
|R34S64NPUZOS90|          5|           23|         24|   N|                N|
|R35O3GV3HZUX8B|          5|           58|         63|   N|                N|
|R2AZAMZCEUOKQT|          1|           29|         40|   N|                Y|
|R103RUVEH5YWQ4|          1|           69|         90|   N|     

In [7]:
# 2) Filter the new DataFrame or table created in Step 1 and create a new DataFrame or table to retrieve all
# the rows where the number of helpful_votes divided by total_votes is equal to or greater than 50%.
helpful_votes_df = vine_df_filtered.filter((vine_df_filtered.helpful_votes/vine_df_filtered.total_votes) >=0.5)
helpful_votes_df.show(20)


+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RV4UH0OWW97K4|          5|           19|         22|   N|                N|
| RG6GT6G9WNYT1|          5|           22|         22|   N|                N|
| RFBXE3Q3GYQYO|          5|           68|         70|   N|                Y|
|R34S64NPUZOS90|          5|           23|         24|   N|                N|
|R35O3GV3HZUX8B|          5|           58|         63|   N|                N|
|R2AZAMZCEUOKQT|          1|           29|         40|   N|                Y|
|R103RUVEH5YWQ4|          1|           69|         90|   N|                Y|
|R2QNPQLXMCXOH3|          5|           35|         37|   N|                Y|
| R1JENRRBQ1YMY|          5|           35|         36|   N|                N|
|R2NS0HPQ2DD5XU|          1|           35|         45|   N|     

In [8]:
# 3) Filter the helpful_votes_df and create a new DataFrame or table that retrieves all the rows where
# a review was written as part of the Vine program (paid)
paid_reviews_df = helpful_votes_df.filter("vine == 'Y'")
paid_reviews_df.show(20)

+---------+-----------+-------------+-----------+----+-----------------+
|review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+---------+-----------+-------------+-----------+----+-----------------+
+---------+-----------+-------------+-----------+----+-----------------+



In [18]:
# 4.0) Filter the helpful_votes_df  and create a new DataFrame or table that retrieves all the rows where
# a review was written as part of the Vine program (unpaid)
unpaid_reviews_df = helpful_votes_df.filter("vine == 'N'")
unpaid_reviews_df.show(20)

1563

In [21]:
# 4.1) Test for mistakes of total vine == 'N' in orignal database and against number for row. 
df_test_1 = df.groupBy("vine").count().show()
df_test_1

+----+------+
|vine| count|
+----+------+
|   N|145431|
+----+------+



In [22]:
# 4.2) Test for mistakes of total vine == 'N' in orignal database and against number for row. 
df_test_2 = df.filter("vine == 'Y'").count()
df_test_2

0

In [11]:
# Determine the total number of reviews, the number of 5-star reviews, and the percentage of 5-star reviews
# for the two types of review (paid vs unpaid).

# Total number of reviews (paid)
paid_review_total_count= paid_reviews_df.count()
paid_review_total_count

0

In [12]:
# Number of 5-star reviews (paid)
paid_review_five_star_count = paid_reviews_df.filter("star_rating==5").count()
paid_review_five_star_count

0

In [13]:
# Percentage of 5-star reviews (paid)
# NOTE: df_test above shows there are not paid review in the dataframe example, code is here displayed for example,
# did not run to avoide dividing by 0

# paid_review_percentage_five_star = (paid_review_five_star_count/paid_review_total_count) * 100
# paid_review_percentage_five_star

In [14]:
# Determine the total number of reviews, the number of 5-star reviews, and the percentage of 5-star reviews
# for the two types of review (paid vs unpaid).

# Total number of reviews (unpaid)
unpaid_review_total_count = unpaid_reviews_df.count()
unpaid_review_total_count

1563

In [15]:
# Number of 5-star reviews (unpaid)
unpaid_review_five_star_count = unpaid_reviews_df.filter("star_rating==5").count()
unpaid_review_five_star_count

582

In [16]:
# Percentage of 5-star reviews (unpaid)
unpaid_review_percentage_five_star = (unpaid_review_five_star_count/unpaid_review_total_count) * 100
unpaid_review_percentage_five_star

37.23608445297505